In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from google.colab import drive
import librosa
import h5py
# Load the TensorBoard notebook extension
!pip install -q pesq
%load_ext tensorboard
%tensorflow_version 2.x
import datetime, os
from tqdm import tqdm
from pesq import pesq
import librosa
import numpy as np
from scipy.signal import resample_poly
import scipy
drive.mount('/content/drive')

     |████████████████████████████████| 81kB 2.2MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Import the Evaluation Kit

In [3]:
EPS = np.finfo("float").eps
def _resample_window_oct(p, q):
    """Port of Octave code to Python"""

    gcd = np.gcd(p, q)
    if gcd > 1:
        p /= gcd
        q /= gcd

    # Properties of the antialiasing filter
    log10_rejection = -3.0
    stopband_cutoff_f = 1. / (2 * max(p, q))
    roll_off_width = stopband_cutoff_f / 10

    # Determine filter length
    rejection_dB = -20 * log10_rejection
    L = np.ceil((rejection_dB - 8) / (28.714 * roll_off_width))

    # Ideal sinc filter
    t = np.arange(-L, L + 1)
    ideal_filter = 2 * p * stopband_cutoff_f \
        * np.sinc(2 * stopband_cutoff_f * t)

    # Determine parameter of Kaiser window
    if (rejection_dB >= 21) and (rejection_dB <= 50):
        beta = 0.5842 * (rejection_dB - 21)**0.4 \
            + 0.07886 * (rejection_dB - 21)
    elif rejection_dB > 50:
        beta = 0.1102 * (rejection_dB - 8.7)
    else:
        beta = 0.0

    # Apodize ideal filter response
    h = np.kaiser(2 * L + 1, beta) * ideal_filter

    return h
def resample_oct(x, p, q):
    """Resampler that is compatible with Octave"""
    h = _resample_window_oct(p, q)
    window = h / np.sum(h)
    return resample_poly(x, p, q, window=window)
def thirdoct(fs, nfft, num_bands, min_freq):
    """ Returns the 1/3 octave band matrix and its center frequencies
    # Arguments :
        fs : sampling rate
        nfft : FFT size
        num_bands : number of 1/3 octave bands
        min_freq : center frequency of the lowest 1/3 octave band
    # Returns :
        obm : Octave Band Matrix
        cf : center frequencies
    """
    f = np.linspace(0, fs, nfft + 1)
    f = f[:int(nfft/2) + 1]
    k = np.array(range(num_bands)).astype(float)
    cf = np.power(2. ** (1. / 3), k) * min_freq
    freq_low = min_freq * np.power(2., (2 * k -1 ) / 6)
    freq_high = min_freq * np.power(2., (2 * k + 1) / 6)
    obm = np.zeros((num_bands, len(f))) # a verifier

    for i in range(len(cf)):
        # Match 1/3 oct band freq with fft frequency bin
        f_bin = np.argmin(np.square(f - freq_low[i]))
        freq_low[i] = f[f_bin]
        fl_ii = f_bin
        f_bin = np.argmin(np.square(f - freq_high[i]))
        freq_high[i] = f[f_bin]
        fh_ii = f_bin
        # Assign to the octave band matrix
        obm[i, fl_ii:fh_ii] = 1
    return obm, cf
def stft(x, win_size, fft_size, overlap=4):
    """ Short-time Fourier transform for real 1-D inputs
    # Arguments
        x : 1D array, the waveform
        win_size : integer, the size of the window and the signal frames
        fft_size : integer, the size of the fft in samples (zero-padding or not)
        overlap: integer, number of steps to make in fftsize
    # Returns
        stft_out : 2D complex array, the STFT of x.
    """
    hop = int(win_size / overlap)
    w = scipy.hanning(win_size + 2)[1: -1]  # = matlab.hanning(win_size)
    stft_out = np.array([np.fft.rfft(w * x[i:i + win_size], n=fft_size)
                        for i in range(0, len(x) - win_size, hop)])
    return stft_out
def remove_silent_frames(x, y, dyn_range, framelen, hop):
    """ Remove silent frames of x and y based on x
    A frame is excluded if its energy is lower than max(energy) - dyn_range
    The frame exclusion is based solely on x, the clean speech signal
    # Arguments :
        x : array, original speech wav file
        y : array, denoised speech wav file
        dyn_range : Energy range to determine which frame is silent
        framelen : Window size for energy evaluation
        hop : Hop size for energy evaluation
    # Returns :
        x without the silent frames
        y without the silent frames (aligned to x)
    """
    # Compute Mask
    w = scipy.hanning(framelen + 2)[1:-1]

    x_frames = np.array(
        [w * x[i:i + framelen] for i in range(0, len(x) - framelen, hop)])
    y_frames = np.array(
        [w * y[i:i + framelen] for i in range(0, len(x) - framelen, hop)])

    # Compute energies in dB
    x_energies = 20 * np.log10(np.linalg.norm(x_frames, axis=1) + EPS)

    # Find boolean mask of energies lower than dynamic_range dB
    # with respect to maximum clean speech energy frame
    mask = (np.max(x_energies) - dyn_range - x_energies) < 0

    # Remove silent frames by masking
    x_frames = x_frames[mask]
    y_frames = y_frames[mask]

    # init zero arrays to hold x, y with silent frames removed
    n_sil = (len(x_frames) - 1) * hop + framelen
    x_sil = np.zeros(n_sil)
    y_sil = np.zeros(n_sil)

    for i in range(x_frames.shape[0]):
        x_sil[range(i * hop, i * hop + framelen)] += x_frames[i, :]
        y_sil[range(i * hop, i * hop + framelen)] += y_frames[i, :]

    return x_sil, y_sil
def vect_two_norm(x, axis=-1):
    """ Returns an array of vectors of norms of the rows of matrices from 3D array """
    return np.sum(np.square(x), axis=axis, keepdims=True)
def row_col_normalize(x):
    """ Row and column mean and variance normalize an array of 2D segments """
    # Row mean and variance normalization
    x_normed = x + EPS * np.random.standard_normal(x.shape)
    x_normed -= np.mean(x_normed, axis=-1, keepdims=True)
    x_inv = 1. / np.sqrt(vect_two_norm(x_normed))
    x_diags = np.array(
        [np.diag(x_inv[i].reshape(-1)) for i in range(x_inv.shape[0])])
    x_normed = np.matmul(x_diags, x_normed)
    # Column mean and variance normalization
    x_normed += + EPS * np.random.standard_normal(x_normed.shape)
    x_normed -= np.mean(x_normed, axis=1, keepdims=True)
    x_inv = 1. / np.sqrt(vect_two_norm(x_normed, axis=1))
    x_diags = np.array(
        [np.diag(x_inv[i].reshape(-1)) for i in range(x_inv.shape[0])])
    x_normed = np.matmul(x_normed, x_diags)
    return x_normed
# Constant definition
FS = 10000                          # Sampling frequency
N_FRAME = 256                       # Window support
NFFT = 512                          # FFT Size
NUMBAND = 15                        # Number of 13 octave band
MINFREQ = 150                       # Center frequency of 1st octave band (Hz)
OBM, CF = thirdoct(FS, NFFT, NUMBAND, MINFREQ)  # Get 1/3 octave band matrix
N = 30                              # N. frames for intermediate intelligibility
BETA = -15.                         # Lower SDR bound
DYN_RANGE = 40                      # Speech dynamic range
def stoi(x, y, fs_sig, extended=False):
    """ Short term objective intelligibility
    Computes the STOI (See [1][2]) of a denoised signal compared to a
    clean signal, The output is expected to have a monotonic
    relation with the subjective speech-intelligibility, where a higher d
    denotes better speech intelligibility
    # Arguments
        x : clean original speech
        y : denoised speech
        fs_sig : sampling rate of x and y
        extended : Boolean, whether to use the extended STOI described in [3]
    # Returns
        Short time objective intelligibility measure between clean and denoised
        speech
    # Raises
        AssertionError : if x and y have different lengths
    # Reference
        [1] C.H.Taal, R.C.Hendriks, R.Heusdens, J.Jensen 'A Short-Time
            Objective Intelligibility Measure for Time-Frequency Weighted Noisy
            Speech', ICASSP 2010, Texas, Dallas.
        [2] C.H.Taal, R.C.Hendriks, R.Heusdens, J.Jensen 'An Algorithm for
            Intelligibility Prediction of Time-Frequency Weighted Noisy Speech',
            IEEE Transactions on Audio, Speech, and Language Processing, 2011.
        [3] Jesper Jensen and Cees H. Taal, 'An Algorithm for Predicting the
            Intelligibility of Speech Masked by Modulated Noise Maskers',
            IEEE Transactions on Audio, Speech and Language Processing, 2016.
    """
    if x.shape != y.shape:
        raise Exception('x and y should have the same length,' +
                        'found {} and {}'.format(x.shape, y.shape))

    # Resample is fs_sig is different than fs
    if fs_sig != FS:
        x = resample_oct(x, FS, fs_sig)
        y = resample_oct(y, FS, fs_sig)

    # Remove silent frames
    x, y = remove_silent_frames(x, y, DYN_RANGE, N_FRAME, int(N_FRAME/2))

    # Take STFT
    x_spec = stft(x, N_FRAME, NFFT, overlap=2).transpose()
    y_spec = stft(y, N_FRAME, NFFT, overlap=2).transpose()

    # Apply OB matrix to the spectrograms as in Eq. (1)
    x_tob = np.sqrt(np.matmul(OBM, np.square(np.abs(x_spec))))
    y_tob = np.sqrt(np.matmul(OBM, np.square(np.abs(y_spec))))

    # Take segments of x_tob, y_tob
    x_segments = np.array(
        [x_tob[:, m - N:m] for m in range(N, x_tob.shape[1] + 1)])
    y_segments = np.array(
        [y_tob[:, m - N:m] for m in range(N, x_tob.shape[1] + 1)])

    if extended:
        x_n = row_col_normalize(x_segments)
        y_n = row_col_normalize(y_segments)
        return np.sum(x_n * y_n / N) / x_n.shape[0]

    else:
        # Find normalization constants and normalize
        normalization_consts = (
            np.linalg.norm(x_segments, axis=2, keepdims=True) /
            (np.linalg.norm(y_segments, axis=2, keepdims=True) + EPS))
        y_segments_normalized = y_segments * normalization_consts

        # Clip as described in [1]
        clip_value = 10 ** (-BETA / 20)
        y_primes = np.minimum(
            y_segments_normalized, x_segments * (1 + clip_value))

        # Subtract mean vectors
        y_primes = y_primes - np.mean(y_primes, axis=2, keepdims=True)
        x_segments = x_segments - np.mean(x_segments, axis=2, keepdims=True)

        # Divide by their norms
        y_primes /= (np.linalg.norm(y_primes, axis=2, keepdims=True) + EPS)
        x_segments /= (np.linalg.norm(x_segments, axis=2, keepdims=True) + EPS)
        # Find a matrix with entries summing to sum of correlations of vectors
        correlations_components = y_primes * x_segments

        # J, M as in [1], eq.6
        J = x_segments.shape[0]
        M = x_segments.shape[1]

        # Find the mean of all correlations
        d = np.sum(correlations_components) / (J * M)
        return d

### Load the Model

In [4]:
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/ThesisData/Multichannel/Model/4channel_large_v4.h5')

### Evaluation Function Definition

In [8]:
counter = 0
FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
Overlap = round(0.75 * FrameSize)
FFTSize = FrameSize  # FFT window size=FRAMESIZE
FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
NumSegments = 8
noisyMean = 0.082 ; noisyStd  = 0.8 ; cleanMean = 0.24 ; cleanStd  = 1.126
from scipy.io import wavfile
import scipy
def convert_to_audio(outname,stft_magnitude,stft_phase,noisyAudio,fs=8000,):
    # Converting the data back to human language
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap   = round(0.75 * FrameSize)
    FFTSize   = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8
    if noisyAudio == True:
      stft_magnitude_nonNormal = noisyStd*stft_magnitude+noisyMean
    else:
      stft_magnitude_nonNormal = cleanStd*stft_magnitude+cleanMean
    final_STFT = stft_magnitude_nonNormal*np.exp(1j*stft_phase)
    # Audio Converter
    converted_audio = librosa.istft(final_STFT, hop_length=Overlap, win_length=FFTSize,
                  window=scipy.signal.hamming(FrameSize,sym=False))
    wavfile.write(outname, rate=fs, data=converted_audio)
def stft_generator(filename,noisyAudio,fs=8000):
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap = round(0.75 * FrameSize)
    FFTSize = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8

    # Reading the audio data
    audio_data, sr = librosa.load(filename, mono=False, sr=fs)  # if sr=None to read raw sample_rate

    if noisyAudio == False:
        stft_segments= librosa.stft(audio_data, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                              window=scipy.signal.hamming(FrameSize,sym=False))
        # Phase Matrix
        phase_segments = np.transpose(np.angle(stft_segments))
        # Magnitude matrix
        stft_segments = np.transpose(abs(stft_segments))
    else:
        counter = 0
        for audio in audio_data:
            channelNumber = 4
            audio = np.nan_to_num(audio)
            noisySTFT = librosa.stft(audio, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                         window=scipy.signal.hamming(FrameSize,sym=False))
            audio_phase = np.angle(noisySTFT)
            phase_segments = audio_phase
            # Magnitude matrix
            noisySTFT =np.abs(noisySTFT)
            new_noisy_STFT = np.concatenate((noisySTFT[:,0:NumSegments-1], noisySTFT), axis=1)
            # initialize STFT 4 channel structure
            if counter == 0:
                stft_segments = np.zeros((new_noisy_STFT.shape[1] - NumSegments + 1, FrequencyBins, NumSegments,channelNumber))

            for index in range(0, new_noisy_STFT.shape[1] - NumSegments + 1):
                stft_segments[index,:,:,counter] = new_noisy_STFT[:, index:index+NumSegments]
            counter = counter + 1

            # phase on:
            # new_audio_phase = np.concatenate((audio_phase[:, 0:NumSegments - 1], audio_phase), axis=1)
            # phase_segments = np.zeros((new_audio_phase.shape[1] - NumSegments + 1, FrequencyBins, NumSegments))
            # for index in range(0, new_audio_phase.shape[1] - NumSegments + 1):
            #     phase_segments[index, :, :] = new_audio_phase[:, index:index + NumSegments]

    return {'stft': stft_segments, 'phase': phase_segments}

# Creating the magnitude and phase spectrums of noisy signal
def evaluate_pesq(cleaned, noisy, clean, fs=8000,):
  noisy_score   = pesq(fs, clean, noisy, 'nb')
  cleaned_score = pesq(fs, clean[:len(cleaned)], cleaned, 'nb')
  return [noisy_score, cleaned_score]

def evaluate_stoi(cleaned, noisy, clean, fs=8000,):
  noisy_score   = stoi(clean, noisy, fs, extended=False)
  cleaned_score = stoi(clean[:len(cleaned)], cleaned, fs, extended=False)
  return [noisy_score, cleaned_score]
def model_evaluate(data_folder, clean_name, noisy_name):
  stft_segments = stft_generator(data_folder + noisy_name,True)
  clean_stft_segments = stft_generator(data_folder + clean_name,False)
  absSTFT = stft_segments['stft']
  noisyPhase = stft_segments['phase']
  cleanPhase = clean_stft_segments['phase']
  clean_stft_segments = clean_stft_segments['stft']

  # Mean-STD Calculations - Noisy
  noisyMean     = np.mean(absSTFT);
  noisyStd      = np.std(absSTFT);
  absSTFT_normalized = (absSTFT - noisyMean)/noisyStd

  # Mean-STD Calculations - Clean
  cleanMean     = np.mean(clean_stft_segments);
  cleanStd      = np.std(clean_stft_segments);
  clean_absSTFT_normalized = (clean_stft_segments - cleanMean)/cleanStd
  noisySTFT = np.reshape(absSTFT_normalized, (absSTFT_normalized.shape[0],absSTFT_normalized.shape[1], absSTFT_normalized.shape[2], absSTFT_normalized.shape[3]));
  cleanSTFT = np.reshape(clean_absSTFT_normalized, (clean_absSTFT_normalized.shape[0], clean_absSTFT_normalized.shape[1], 1, 1));
  # Evaluation
  cleanedup_audio = model.predict(noisySTFT)
  print('\n# Evaluate on test data')
  model.evaluate(noisySTFT,  cleanSTFT, verbose=1)
  # Writing to the FILE
  final_cleanedup_audio = np.reshape(cleanedup_audio, (noisySTFT.shape[0], noisySTFT.shape[1]));
  final_cleanSTFT = np.reshape(cleanSTFT, (clean_stft_segments.shape[0], clean_stft_segments.shape[1]));
  convert_to_audio(data_folder+ 'CLEANED_v4/cleaned_' + noisy_name, np.transpose(final_cleanedup_audio),noisyPhase,False)

**Running the Code**
The functions are defined above. In the section below, the evaluation takes place.

In [9]:
import pandas as pd
data_folder = '/content/drive/My Drive/Colab Notebooks/ThesisData/Multichannel/Data/4Channel/TestData/FullMixture_5k_4Channel_Fixed_TEST/'
all_sound_files = os.listdir(data_folder)
mixture_files = [i for i in all_sound_files if "_" in i and "wav" in i]
clean_files   = [i for i in all_sound_files if not "_" in i]
noisy_stoi_score = []; noisy_pesq_score = []; mixture_name = []
cleaned_stoi_score = []; cleaned_pesq_score = []
for clean_audio in tqdm(clean_files):
    mixture_number = '_' + clean_audio.split('.')[0][5:] + '_'
    if any(mixture_number in x  for x in mixture_files):
        noisy_name = [item for i, item in enumerate(mixture_files) if mixture_number in item][0]
    else:
        continue
    mixture_name.append(noisy_name)
    # Evaluating the signals
    model_evaluate(data_folder,   clean_audio, noisy_name)
    path_clean   = data_folder +  clean_audio
    path_noisy   = data_folder +  noisy_name
    path_cleaned = data_folder + 'CLEANED_v4/cleaned_' + noisy_name
    clean, fs    = librosa.load(path_clean,   mono=True, sr=8000)
    noisy, fs    = librosa.load(path_noisy,   mono=True, sr=8000)
    cleaned, fs  = librosa.load(path_cleaned, mono=True, sr=8000)
    # Score Evaluation
    pesq_score   = evaluate_pesq(cleaned, noisy, clean)
    stoi_score   = evaluate_stoi(cleaned, noisy, clean)
    # Append Noisy Scores
    noisy_stoi_score.append(stoi_score[0])
    noisy_pesq_score.append(pesq_score[0])
    # Append Cleaned Scores
    cleaned_stoi_score.append(stoi_score[1])
    cleaned_pesq_score.append(pesq_score[1])

scores = {'Mixture Name': mixture_name,
         'Noisy STOI Score': noisy_stoi_score,
         'Cleaned STOI Score': cleaned_stoi_score,
         'Noisy PESQ Score': noisy_pesq_score,
         'Cleaned PESQ Score': cleaned_pesq_score
        }
df = pd.DataFrame(scores, columns= ['Mixture Name', 'Noisy STOI Score','Cleaned STOI Score','Noisy PESQ Score','Cleaned PESQ Score'])

df.to_csv(data_folder+'SCORES/scores_v4.csv', index = False, header=True)

  0%|          | 0/478 [00:00<?, ?it/s]


# Evaluate on test data
3/3 [==============================] - 0s 36ms/step - loss: 0.1031 - mean_squared_error: 0.2924


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
  0%|          | 1/478 [00:01<13:27,  1.69s/it]


# Evaluate on test data
4/4 [==============================] - 0s 51ms/step - loss: 0.1327 - mean_squared_error: 0.5299


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
  0%|          | 2/478 [00:03<14:03,  1.77s/it]

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
 99%|█████████▉| 475/478 [18:19<00:06,  2.21s/it]


# Evaluate on test data
3/3 [==============================] - 0s 30ms/step - loss: 0.0788 - mean_squared_error: 0.2395


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
100%|█████████▉| 476/478 [18:21<00:04,  2.12s/it]


# Evaluate on test data
6/6 [==============================] - 0s 49ms/step - loss: 0.1933 - mean_squared_error: 0.6423


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: scipy.hanning is deprecated and will be removed in SciPy 2.0.0, use numpy.hanning instead
100%|██████████| 478/478 [18:24<00:00,  2.31s/it]
